In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import *
from sklearn.neighbors import NearestNeighbors
import time
import re
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# from cluster0 and cluster1 dfs:
# 1) make dfs with just trip_id, PC1_2d, PC2_2d, Qmodegrouping
# 2) create separate df where qmodegrouping= cars, remove car trips
# 3) onehotencode dfs
# 4) scale dfs

In [33]:
#  1
cluster0 = pd.read_csv('cluster0_df.csv', index_col=0)
cluster1 = pd.read_csv('cluster1_df.csv', index_col=0)

In [34]:
cluster0.head()

,trip_id,uniqueid,qday,qday1typical,qborough_home,qsurveyzone_home,qtripdaytime,qborough_start,surveyzone_start,qday1tripend,...,qprecipitation,qcitibike,qday1triptravelcode_sp,qmodegrouping,qsustainablemode,allwt,qpurposerecode,Cluster,PC1_2d,PC2_2d
0,3101674,101674,Sunday,Very typical,Queens,Middle Queens,NOON,Queens,Outer Queens,Home,...,0.00,No,Citi Bike,Bike,Yes,2.269123,School,0,-0.600979,-0.165431
1,11202717,10202717,Saturday,Very typical,Queens,Middle Queens,AM,Queens,Middle Queens,Home,...,0.25,No,Personal bicycle,Bike,Yes,0.868687,Commute to/from work,0,0.164231,-0.458575
2,12202717,10202717,Saturday,Very typical,Queens,Middle Queens,NOON,Middle Queens,Middle Queens,Park/Recreational area/Gym,...,0.25,No,Personal bicycle,Bike,Yes,0.868687,Social/recreation,0,0.144638,-0.281665
3,51200330,50200330,Thursday,Very typical,Queens,Middle Queens,AM,Queens,Middle Queens,Park/Recreational area/Gym,...,0.00,No,Personal bicycle,Bike,Yes,2.433119,Social/recreation,0,-0.439627,-0.127318
4,52200330,50200330,Thursday,Very typical,Queens,Middle Queens,AM,Queens,Middle Queens,Home,...,0.00,No,Personal bicycle,Bike,Yes,2.433119,Social/recreation,0,-0.422489,0.061138


In [41]:
min_cluster0_df = cluster0[['trip_id', 'PC1_2d', 'PC2_2d', 'qmodegrouping']]


In [42]:
min_cluster0_df.tail()

,trip_id,PC1_2d,PC2_2d,qmodegrouping
7782,55202898,-0.332950,4.937238,Car
7783,55202919,0.040202,3.677752,Bus
7784,55202932,-0.814321,3.362223,Car
7785,55203010,-0.137633,5.628656,Subway
7787,56200023,-0.099164,5.272096,Walk


In [46]:
df0 = min_cluster0_df.replace('Car', np.nan).dropna()

In [48]:
df0.tail()

,trip_id,PC1_2d,PC2_2d,qmodegrouping
7779,55202675,-0.085413,3.848885,Subway
7781,55202779,-0.231592,3.955031,Bus
7783,55202919,0.040202,3.677752,Bus
7785,55203010,-0.137633,5.628656,Subway
7787,56200023,-0.099164,5.272096,Walk


In [43]:
min_cluster1_df = cluster1[['trip_id', 'PC1_2d', 'PC2_2d', 'qmodegrouping']]



In [44]:
min_cluster1_df.tail()

,trip_id,PC1_2d,PC2_2d,qmodegrouping
7345,53202334,-0.217588,2.425244,Car
7346,53202369,-0.186420,1.284795,Car
7602,54201972,-0.128017,3.696877,Car
7723,55200993,4.921119,1.468521,Walk
7786,55203034,-0.272619,5.081466,Subway


In [49]:
df1 = min_cluster1_df.replace('Car', np.nan).dropna()

In [51]:
df1.tail()

,trip_id,PC1_2d,PC2_2d,qmodegrouping
6925,52202562,-0.515103,-0.098558,Subway
6926,52202569,-0.447968,-0.103074,Subway
7343,53202320,-0.345487,0.277337,Walk
7723,55200993,4.921119,1.468521,Walk
7786,55203034,-0.272619,5.081466,Subway


In [52]:
# 3

###one hot encode transit modes
dummies0 = pd.get_dummies(df0)
dummies0.head()

#dress_items_dummies > items_df_dress > dress_scaled
#scaled_df - target variable 

,trip_id,PC1_2d,PC2_2d,qmodegrouping_Bike,qmodegrouping_Bus,qmodegrouping_Commuter Rail,qmodegrouping_Ferry,qmodegrouping_For-Hire Vehicle,qmodegrouping_Other,qmodegrouping_Subway,qmodegrouping_Walk
0,3101674,-0.600979,-0.165431,1,0,0,0,0,0,0,0
1,11202717,0.164231,-0.458575,1,0,0,0,0,0,0,0
2,12202717,0.144638,-0.281665,1,0,0,0,0,0,0,0
3,51200330,-0.439627,-0.127318,1,0,0,0,0,0,0,0
4,52200330,-0.422489,0.061138,1,0,0,0,0,0,0,0


In [53]:
dummies1 = pd.get_dummies(df1)
dummies1.head()

,trip_id,PC1_2d,PC2_2d,qmodegrouping_Bike,qmodegrouping_Bus,qmodegrouping_Commuter Rail,qmodegrouping_Ferry,qmodegrouping_For-Hire Vehicle,qmodegrouping_Other,qmodegrouping_Subway,qmodegrouping_Walk
14,1103836,5.031384,0.161109,1,0,0,0,0,0,0,0
20,3103836,5.067291,0.201096,1,0,0,0,0,0,0,0
22,4103836,5.102120,0.714898,1,0,0,0,0,0,0,0
24,5105414,5.159970,2.142712,1,0,0,0,0,0,0,0
41,15204136,5.060177,3.679616,1,0,0,0,0,0,0,0


In [56]:
# 4
###scale and convert to 
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# dress_scaled = scaler.fit_transform(items_df_dress)

# # Convert into a DataFrame
# scaled_df = pd.DataFrame(dress_scaled, columns=items_df_dress.columns, index = items_df_dress.item_id)
# scaled_df = scaled_df.drop(['item_id'], axis = 1)
# scaled_df.head()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df0_scaled = scaler.fit_transform(dummies0)

# Convert into a DataFrame
scaled_df0 = pd.DataFrame(df0_scaled, columns=dummies0.columns, index = dummies0.trip_id) #maybe not trip_id...
scaled_df0 = scaled_df0.drop(['trip_id'], axis = 1)
scaled_df0.tail()


,PC1_2d,PC2_2d,qmodegrouping_Bike,qmodegrouping_Bus,qmodegrouping_Commuter Rail,qmodegrouping_Ferry,qmodegrouping_For-Hire Vehicle,qmodegrouping_Other,qmodegrouping_Subway,qmodegrouping_Walk
trip_id,,,,,,,,,,
55202675,0.227916,0.648350,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
55202779,0.206586,0.660418,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
55202919,0.246245,0.628894,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
55203010,0.220296,0.850698,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
56200023,0.225910,0.810160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [58]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df1_scaled = scaler.fit_transform(dummies1)

# Convert into a DataFrame
scaled_df1 = pd.DataFrame(df1_scaled, columns=dummies1.columns, index = dummies1.trip_id) #maybe not trip_id...
scaled_df1 = scaled_df1.drop(['trip_id'], axis = 1)
scaled_df1.tail()

,PC1_2d,PC2_2d,qmodegrouping_Bike,qmodegrouping_Bus,qmodegrouping_Commuter Rail,qmodegrouping_Ferry,qmodegrouping_For-Hire Vehicle,qmodegrouping_Other,qmodegrouping_Subway,qmodegrouping_Walk
trip_id,,,,,,,,,,
52202562,0.137165,0.200716,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
52202569,0.147359,0.200204,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
53202320,0.162921,0.243344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
55200993,0.962663,0.378431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
55203034,0.173986,0.788158,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [60]:

#maybe cosine sim?
cos_sim_scaled0 = cosine_similarity(scaled_df0)
cos_sim_scaled1 = cosine_similarity(scaled_df1)

In [61]:

indices0=pd.Series(scaled_df0.index)
indices1=pd.Series(scaled_df1.index)

In [69]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices0 = pd.Series(scaled_df0.index)
#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations0(trip_id, cos_sim = cos_sim_scaled0):
    
    # initializing the empty list of recommended movies
    rec_items = []
    
    # gettin the index of the movie that matches the title
    idx = indices0[indices0 == trip_id].index[0]
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)
    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        rec_items.append(list(scaled_df0.index)[i])
        
    return rec_items

In [72]:
recommendations0(55202675)

[53200539,
 52202606,
 55201693,
 55200093,
 51200886,
 13200849,
 14200517,
 55200504,
 11201148,
 53201637]

In [92]:
min_cluster0_df.ix[(min_cluster0_df.trip_id =='53201637').idxmin(), 'qmodegrouping']

#https://stackoverflow.com/questions/23486083/accessing-another-column-by-value-pandas


/Users/brittanyfowle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


'Bike'

In [ ]:
# min_cluster0_df.ix[(min_cluster0_df.trip_id =='53200539').idxmin(), 'qmodegrouping']


In [80]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.loc.html

#min_cluster0_df['53200539'].loc['qmodegrouping']

In [74]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices1 = pd.Series(scaled_df1.index)
#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations1(trip_id, cos_sim = cos_sim_scaled1):
    
    # initializing the empty list of recommended movies
    rec_items = []
    
    # gettin the index of the movie that matches the title
    idx = indices1[indices1 == trip_id].index[0]
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)
    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        rec_items.append(list(scaled_df1.index)[i])
        
    return rec_items

In [75]:
recommendations1(1103836)

[3103836,
 54203033,
 53203033,
 55203033,
 4103836,
 5105414,
 16204136,
 15204136,
 20204136,
 6105414]

In [96]:
min_cluster1_df.ix[(min_cluster1_df.trip_id =='4103836').idxmin(), 'qmodegrouping']


/Users/brittanyfowle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


'Bike'